# Vlaamse parlement API

Het Vlaamse Parlement stelt de data uit de parlementaire databank ook ter beschikking via een API: https://www.vlaamsparlement.be/nl/parlementair-werk/dossiers/dossiers/open-data en http://ws.vlpar.be/e/opendata/api/.

# Setting up

In [1]:
# show all outputs of cell, not merely of last line (i.e. default of Jupyter Notebook)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd

import csv
import pickle

from collections import defaultdict

import requests

from datetime import datetime
import locale # to allow date parsing for dates in Dutch

from collections import Counter

import matplotlib.pyplot as plt

import copy

In [3]:
# # Set the locale to Dutch
# locale.setlocale(locale.LC_ALL, 'nl_NL')

In [4]:
# Obtain string of current date
today_str = datetime.now().strftime("%Y-%m-%d")
today_str 

'2023-12-23'

In [5]:
# Set base_url of api
base_url = "https://ws.vlpar.be/e/opendata"

The webpage of the API shows some interesting fields:
* `/stats/{commId}/{zj}`: statistieken voor commissie per zittingsjaar
* `/vv/huidige`: Lijst van huidige Vlaamse volksvertegenwoordigers
* `/vv/gewezen`: Lijst van gewezen Vlaamse volksvertegenwoordigers
* `/vv/{persoonId}` Detailgegevens volksvertegenwoordiger
* `/comm/huidige` Commissies van de huidige legislatuur
* `/comm/{commId}` Samenstelling commissie
* `/verg/vorige` Lijst van vorige vergaderingen voor periode
* `/verg/zoek/datums` Lijst van vorige vergaderingen (beperkte data) binnen een zekere periode

# Read in data

In [6]:
def get_endpoint(endpoint: str):
    """
    Return data available at inserted endpoint
    """
    # Make the GET request
    response = requests.get(f"{base_url}{endpoint}")

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()  # Parse JSON response
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    return data

## Members and parties

First we obtain the current members of parliament.

In [7]:
volksvertegenwoordigers_json = get_endpoint("/vv/huidige")

# # Inspect data
# volksvertegenwoordigers_json

# Parse json into dataframe. Visual inspection showed data is stored in 'items' key
volksvertegenwoordigers_df = pd.DataFrame.from_dict(
    pd.json_normalize(volksvertegenwoordigers_json['items']), 
    orient='columns')

In [8]:
# Inspect results
volksvertegenwoordigers_df.head()
volksvertegenwoordigers_df.columns

,volksvertegenwoordiger.deelstaatsenator,volksvertegenwoordiger.fotowebpath,volksvertegenwoordiger.fractie.id,volksvertegenwoordiger.fractie.kleur,volksvertegenwoordiger.fractie.logo,volksvertegenwoordiger.fractie.naam,volksvertegenwoordiger.fractie.zetel-aantal,volksvertegenwoordiger.id,volksvertegenwoordiger.is-huidige-vv,volksvertegenwoordiger.kieskring,volksvertegenwoordiger.link,volksvertegenwoordiger.naam,volksvertegenwoordiger.voornaam,volksvertegenwoordiger.zetel
0,False,https://docs.vlaamsparlement.be/pfile?id=1695546,1279907,83de62,https://docs.vlaamsparlement.be/pfile?id=1866773,Groen,14,4544,J,Kieskring Antwerpen,[{'href': 'https://ws.vlpar.be/e/opendata/vv/4...,Aerts,Staf,127
1,False,https://docs.vlaamsparlement.be/pfile?id=1483067,1279907,83de62,https://docs.vlaamsparlement.be/pfile?id=1866773,Groen,14,4381,J,Kieskring Antwerpen,[{'href': 'https://ws.vlpar.be/e/opendata/vv/4...,Almaci,Meyrem,25
2,True,https://docs.vlaamsparlement.be/pfile?id=1483322,-1,787878,NaN,Onafhankelijke,2,4382,J,Kieskring Brussel-Hoofdstad,[{'href': 'https://ws.vlpar.be/e/opendata/vv/4...,Ampe,Els,130
3,False,https://docs.vlaamsparlement.be/pfile?id=1482456,1279894,FF2900,https://docs.vlaamsparlement.be/pfile?id=1737107,Vooruit,13,4383,J,Kieskring Antwerpen,[{'href': 'https://ws.vlpar.be/e/opendata/vv/4...,Anaf,Hannes,57
4,False,https://docs.vlaamsparlement.be/pfile?id=1478733,1279907,83de62,https://docs.vlaamsparlement.be/pfile?id=1866773,Groen,14,4052,J,Kieskring Antwerpen,[{'href': 'https://ws.vlpar.be/e/opendata/vv/4...,Annouri,Imade,59


Index(['volksvertegenwoordiger.deelstaatsenator',
       'volksvertegenwoordiger.fotowebpath',
       'volksvertegenwoordiger.fractie.id',
       'volksvertegenwoordiger.fractie.kleur',
       'volksvertegenwoordiger.fractie.logo',
       'volksvertegenwoordiger.fractie.naam',
       'volksvertegenwoordiger.fractie.zetel-aantal',
       'volksvertegenwoordiger.id', 'volksvertegenwoordiger.is-huidige-vv',
       'volksvertegenwoordiger.kieskring', 'volksvertegenwoordiger.link',
       'volksvertegenwoordiger.naam', 'volksvertegenwoordiger.voornaam',
       'volksvertegenwoordiger.zetel'],
      dtype='object')

This dataframe contains the relevant party names and their corresponding colour. We can use this later when visualising, so we obtain this and store it.

In [9]:
partij_kleur_dict = {} # Create empty dict to fill

# Iterate over each member and if its party and colour not yet in dict, include
for kleur, partij in zip(volksvertegenwoordigers_df["volksvertegenwoordiger.fractie.kleur"],
    volksvertegenwoordigers_df["volksvertegenwoordiger.fractie.naam"]):
    if partij not in partij_kleur_dict.keys():
        partij_kleur_dict[partij] = kleur

# Inspect results
partij_kleur_dict

{'Groen': '83de62',
 'Onafhankelijke': '787878',
 'Vooruit': 'FF2900',
 'Vlaams Belang': 'ffe500',
 'cd&v': 'f5822a',
 'N-VA': 'ffac12',
 'Open Vld': '003d6d',
 'PVDA': 'AA050E'}

In [10]:
#Export results to pickle file
with open(f'../data/partij_kleur_dict.pkl', 'wb') as file:
    pickle.dump(partij_kleur_dict, file)

Then we obtain a dictionary that groups all members (and their ID) of each party, and a dictionary that maps each members and its ID to its party.

In [11]:
# Create empty dicts
fracties_dict = {}
parlementsleden_all_dict = {}

#Iterate over all rows and store fractie, id and voornaam and naam in dicts
for index, row in volksvertegenwoordigers_df.iterrows():
    fractie = row["volksvertegenwoordiger.fractie.naam"]
    parlementslid_id = row["volksvertegenwoordiger.id"]
    voornaam_en_naam = row["volksvertegenwoordiger.voornaam"] + " " + row["volksvertegenwoordiger.naam"]
    
    if fractie not in fracties_dict:
        fracties_dict[fractie] = []
    fracties_dict[fractie].append([voornaam_en_naam, parlementslid_id])
    
    parlementsleden_all_dict[parlementslid_id] = [voornaam_en_naam, fractie]

In [12]:
# Inspect results
fracties_dict
parlementsleden_all_dict

{'Groen': [['Staf Aerts', 4544],
  ['Meyrem Almaci', 4381],
  ['Imade Annouri', 4052],
  ['Stijn Bex', 4385],
  ['Johan Danen', 4069],
  ['Ann De Martelaer', 1781],
  ['Celia Groothedde', 4398],
  ['Elisabeth Meuleman', 3173],
  ['An Moerenhout', 4058],
  ['Björn Rzoska', 3920],
  ['Mieke Schauvliege', 4409],
  ['Chris Steenwegen', 4414],
  ['Tine Van den Brande', 4419],
  ['Jeremie Vaneeckhout', 4426]],
 'Onafhankelijke': [['Els Ampe', 4382], ['Sihame El Kaouakibi', 4396]],
 'Vooruit': [['Hannes Anaf', 4383],
  ['Kurt De Loor', 2791],
  ['Hannelore Goeman', 4397],
  ['Emine Gül Isci', 4598],
  ['Annick Lambrecht', 4402],
  ['Els Robeyns', 3221],
  ['Katia Segers', 4087],
  ['Bruno Tobback', 1882],
  ['Steve Vandenberghe', 4107],
  ['Freya Van den Bossche', 3458],
  ['Ludwig Vandenhove', 4423],
  ['Thijs Verbeurgt', 4583],
  ['Maxim Veys', 4428]],
 'Vlaams Belang': [['Roosmarijn Beckers', 4384],
  ['Adeline Blancquaert', 4386],
  ['Filip Brusselmans', 4387],
  ['Yves Buysse', 4388],
  

{4544: ['Staf Aerts', 'Groen'],
 4381: ['Meyrem Almaci', 'Groen'],
 4382: ['Els Ampe', 'Onafhankelijke'],
 4383: ['Hannes Anaf', 'Vooruit'],
 4052: ['Imade Annouri', 'Groen'],
 4384: ['Roosmarijn Beckers', 'Vlaams Belang'],
 4385: ['Stijn Bex', 'Groen'],
 4386: ['Adeline Blancquaert', 'Vlaams Belang'],
 3430: ['Robrecht Bothuyne', 'cd&v'],
 3426: ['Karin Brouwers', 'cd&v'],
 4387: ['Filip Brusselmans', 'Vlaams Belang'],
 4388: ['Yves Buysse', 'Vlaams Belang'],
 4389: ['Allessia Claes', 'N-VA'],
 4390: ['Bart Claes', 'Vlaams Belang'],
 4444: ['Arnout Coel', 'N-VA'],
 4446: ['Steven Coenegrachts', 'Open Vld'],
 4068: ['Cathy Coudyser', 'N-VA'],
 4069: ['Johan Danen', 'Groen'],
 4070: ['Koen Daniëls', 'N-VA'],
 2838: ['Johan Deckmyn', 'Vlaams Belang'],
 4391: ['Inez De Coninck', 'N-VA'],
 3416: ['Jean-Jacques De Gucht', 'Open Vld'],
 2791: ['Kurt De Loor', 'Vooruit'],
 1781: ['Ann De Martelaer', 'Groen'],
 4432: ['Immanuel De Reuse', 'Vlaams Belang'],
 2793: ['Annick De Ridder', 'N-VA'],


In [13]:
#Export results
# Save to CSV
with open(f'../data/fracties.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in fracties_dict.items():
        writer.writerow([key, value])

with open(f'../data/parlementsleden.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in parlementsleden_all_dict.items():
        writer.writerow([key, value])

# Save to pickle file
with open(f'../data/fracties.pkl', 'wb') as file:
    pickle.dump(fracties_dict, file)

with open(f'../data/parlementsleden.pkl', 'wb') as file:
    pickle.dump(parlementsleden_all_dict, file)

380

70

354

640

515

...

35

34

34

42

33

## Commissions

Then we obtain the current commissions. 

In [14]:
commissies_json = get_endpoint("/comm/huidige")

# # Inspect data
# commissies_json

# Parse json into dataframe. Visual inspection showed data is stored in 'items' key
commissies_pd = pd.DataFrame.from_dict(pd.json_normalize(commissies_json['items']), orient='columns')

In [15]:
# Inspect results
commissies_pd.head()
commissies_pd.columns
# commissies_pd["commissie.titel"]

,commissie.afkorting,commissie.datumtot,commissie.datumvan,commissie.id,commissie.link,commissie.titel
0,VKO,2022-07-13T00:00:59+0200,2022-03-23T14:00:00+0100,1621329,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,Onderzoekscommissie naar de veiligheid in de k...
1,VEKP,NaN,2022-03-30T14:00:00+0200,1622680,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,Commissie Vlaams Energie- en Klimaatplan
2,WIZ,2022-07-04T23:45:59+0200,2020-12-16T14:00:00+0100,1456916,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,Werkgroep Institutionele Zaken
3,FIN,NaN,2019-10-09T14:00:00+0200,1332937,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,"Commissie voor Algemeen Beleid, Financiën, Beg..."
4,BIN,NaN,2019-10-09T14:00:00+0200,1332966,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,"Commissie voor Binnenlands Bestuur, Gelijke Ka..."


Index(['commissie.afkorting', 'commissie.datumtot', 'commissie.datumvan',
       'commissie.id', 'commissie.link', 'commissie.titel'],
      dtype='object')

In [16]:
#Only keep relevant columns
commissions_overview_df = commissies_pd[["commissie.id", "commissie.titel", "commissie.link"]]

# Inspect results
commissions_overview_df                                

,commissie.id,commissie.titel,commissie.link
0,1621329,Onderzoekscommissie naar de veiligheid in de k...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...
1,1622680,Commissie Vlaams Energie- en Klimaatplan,[{'href': 'https://ws.vlpar.be/e/opendata/comm...
2,1456916,Werkgroep Institutionele Zaken,[{'href': 'https://ws.vlpar.be/e/opendata/comm...
3,1332937,"Commissie voor Algemeen Beleid, Financiën, Beg...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...
4,1332966,"Commissie voor Binnenlands Bestuur, Gelijke Ka...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...
5,1332996,Commissie voor Brussel en de Vlaamse Rand en D...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...
6,1333030,"Commissie voor Buitenlands Beleid, Europese Aa...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...
7,1333053,"Commissie voor Cultuur, Jeugd, Sport en Media",[{'href': 'https://ws.vlpar.be/e/opendata/comm...
8,1333088,"Commissie voor Economie, Werk, Sociale Economi...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...
9,1333137,"Commissie voor Landbouw, Visserij en Plattelan...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...


Then we obtain more details for each commission, using information obtained when parsing the current commissions. We store this in a dict: `commissies_samenstelling_dict`.

In [17]:
# Parse information about each commission and store in dict
commissies_samenstelling_dict = {}

for commId in commissies_pd["commissie.id"]:
    commissies_samenstelling_dict[commId] = get_endpoint(f"/comm/{commId}")

In [18]:
# Inspect results
commissies_samenstelling_dict

{1621329: {'afkorting': 'VKO',
  'commissiesecretaris': [{'aanspreking': 'De heer',
    'email': 'nico.lemaire@vlaamsparlement.be',
    'id': 2026,
    'naam': 'Lemaire',
    'telnr': ['02 552 1343'],
    'voornaam': 'Nico'}],
  'datum-van-samenstelling': '2022-07-13T00:00:59+0200',
  'datumtot': '2022-07-13T00:00:59+0200',
  'datumvan': '2022-03-23T14:00:00+0100',
  'disclaimer': '',
  'functie': [],
  'id': 1621329,
  'naam': 'Onderzoekscommissie naar de veiligheid in de kinderopvang'},
 1622680: {'afkorting': 'VEKP',
  'commissiesecretaris': [],
  'datum-van-samenstelling': '2023-12-23T18:45:58+0100',
  'datumvan': '2022-03-30T14:00:00+0200',
  'disclaimer': '',
  'functie': [{'lid': [{'aanspreking': 'De heer',
      'fotowebpath': 'https://docs.vlaamsparlement.be/pfile?id=1964484',
      'fractie': {'id': 1279896,
       'kleur': 'ffac12',
       'logo': 'https://docs.vlaamsparlement.be/pfile?id=1737102',
       'naam': 'N-VA',
       'zetel-aantal': 35},
      'id': 4077,
      'i

Inspection of the results shows that for each commission (identified by its id), some information is provide, such as: `afkorting`, `naam`, `commissiesecretaris`, as well as `functie`. This last tag contains the various possible functions for members in the commission (such as 'voorzitter', 'vast lid', 'plaatsvervangend lid'), and the information of those members.  Hence, we can use this tag to extract the members of all commissions and store it in a dict `commission_members_dict`. We also assess which different functions occur over all commissions (see below). Based on these functions, we modify the cell below to store all relevant functions (e.g. if only 'ondervoorzitter' and 'secretary': for consistency store as 'eerste ondervoorzitter' en 'tweede ondervoorzitter').  

In [19]:
# # Create empty dictionary to store members in
# commission_members_dict = {}

# Create empty columns in dataframe
commissions_overview_df["voorzitter"] = ""
commissions_overview_df["eerste ondervoorzitter"] = ""
commissions_overview_df["tweede ondervoorzitter"] = ""
commissions_overview_df["derde ondervoorzitter"] = ""
commissions_overview_df["vierde ondervoorzitter"] = ""
commissions_overview_df["vaste leden"] = ""
commissions_overview_df["plaatsvervangende leden"] = "" 
commissions_overview_df["toegevoegde leden"] = ""

# Create empty list to store all different functions (to later on create set of to find distinct functions)
diff_functions_list = []

# Iterate over all different commissions
for index_overview, row_overview in commissions_overview_df.iterrows():
    commissie_id = row_overview["commissie.id"]
    commissie_naam = row_overview["commissie.titel"]
    
    # Check if the commissie_id exists in commissies_samenstelling_dict (if not doing so: yields error when nan as key)
    if commissie_id in commissies_samenstelling_dict:
        # store information on members, i.e. those in tag 'functie'
        members = commissies_samenstelling_dict[commissie_id]['functie']
        
        # create empty list of names and functions for each new commission
        names = []  
        functions = []  

        # Iterate over each function / group of members
        for member_group in members:
            function_name = member_group['naam']
            # Go one level deeper to assess all members of the group / all those with this function
            for person in member_group['lid']:
                first_name = person['voornaam']
                last_name = person['naam']
                full_name = f"{first_name} {last_name}"
                parlementslid_id = person['id']

                names.append(full_name)
                functions.append(function_name)

        # Create defaultdict to group members by party, fill it, and retransfer to normal dict
        parlementsleden_groupeddict = defaultdict(list)
        for name, func in zip(names, functions):
            parlementsleden_groupeddict[func].append(name)
        parlementsleden_dict = dict(parlementsleden_groupeddict)
        
        # Append all functions to list to later on check all possible functions
        for spec_function in parlementsleden_dict.keys():
            diff_functions_list.append(spec_function) 
            
        # Fill dataframe with all members performing specific functions
        # include 'or []' to address cases where no such key found. Else, will yield None, wich cannot be concatenated with other None
        # explicitly cast to list when using 'or []' since when you use the + operator with lists, it concatenates them as expected. 
        # However, in cases where there's a single item, it might sometimes return a tuple, especially when using the or [] approach to handle None values.
        commissions_overview_df.at[index_overview, "voorzitter"] = parlementsleden_dict.get('voorzitter')
        commissions_overview_df.at[index_overview, "eerste ondervoorzitter"] = list((parlementsleden_dict.get('eerste ondervoorzitter') or []) + 
                                                                                    (parlementsleden_dict.get('ondervoorzitter') or []))
        commissions_overview_df.at[index_overview, "tweede ondervoorzitter"] = list((parlementsleden_dict.get('tweede ondervoorzitter') or []) + 
                                                                                    (parlementsleden_dict.get('secretaris') or []))
        commissions_overview_df.at[index_overview, "derde ondervoorzitter"] = list((parlementsleden_dict.get('derde ondervoorzitter') or []) + 
                                                                                   (parlementsleden_dict.get('verslaggever') or []))
        commissions_overview_df.at[index_overview, "vierde ondervoorzitter"] = parlementsleden_dict.get('vierde ondervoorzitter')
        # Add to 'vaste leden' also anyone with important functions since they are in essence 'vast lid +'.
        # Also add toegevoegd lid since they also provide information about absence. 
        commissions_overview_df.at[index_overview, "vaste leden"] = list((parlementsleden_dict.get('voorzitter') or []) +
                                                                         (parlementsleden_dict.get('eerste ondervoorzitter') or []) + 
                                                                         (parlementsleden_dict.get('ondervoorzitter') or []) + 
                                                                         (parlementsleden_dict.get('tweede ondervoorzitter') or []) + 
                                                                         (parlementsleden_dict.get('secretaris') or []) + 
                                                                         (parlementsleden_dict.get('derde ondervoorzitter') or []) +
                                                                         (parlementsleden_dict.get('verslaggever') or []) + 
                                                                         (parlementsleden_dict.get('vierde ondervoorzitter') or []) +
                                                                         (parlementsleden_dict.get('vast lid') or []) + 
                                                                         (parlementsleden_dict.get('lid') or []) + 
                                                                         (parlementsleden_dict.get('toegevoegd lid') or [])                                                                      
                                                                        )
        commissions_overview_df.at[index_overview, "plaatsvervangende leden"] = parlementsleden_dict.get('plaatsvervangend lid')
        commissions_overview_df.at[index_overview, "toegevoegde leden"] = parlementsleden_dict.get('toegevoegd lid')
#         # Assess which commissions have 'verslaggever' as function
#         # Onderzoekscommissie PFAS-PFOS apparantly has a 'verslaggever' 
#         if 'verslaggever' in parlementsleden_dict.keys():
#             print (commissie_naam)

/tmp/ipykernel_25414/2428549152.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commissions_overview_df["voorzitter"] = ""
/tmp/ipykernel_25414/2428549152.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commissions_overview_df["eerste ondervoorzitter"] = ""


In [20]:
# Inspect all possible functions in various commissions
set(diff_functions_list)

# for key in commission_members_dict.keys():
#     if commission_members_dict[key][]

{'derde ondervoorzitter',
 'eerste ondervoorzitter',
 'lid',
 'ondervoorzitter',
 'plaatsvervangend lid',
 'secretaris',
 'toegevoegd lid',
 'tweede ondervoorzitter',
 'vast lid',
 'verslaggever',
 'vierde ondervoorzitter',
 'voorzitter'}

In [21]:
# # Inspect results
# commissions_overview_df.iloc[1]['vaste leden']
commissions_overview_df

,commissie.id,commissie.titel,commissie.link,voorzitter,eerste ondervoorzitter,tweede ondervoorzitter,derde ondervoorzitter,vierde ondervoorzitter,vaste leden,plaatsvervangende leden,toegevoegde leden
0,1621329,Onderzoekscommissie naar de veiligheid in de k...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,None,[],[],[],None,[],None,None
1,1622680,Commissie Vlaams Energie- en Klimaatplan,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Andries Gryffroy],[Bruno Tobback],[Robrecht Bothuyne],[],None,"[Andries Gryffroy, Bruno Tobback, Robrecht Bot...","[Adeline Blancquaert, Bart Claes, Inez De Coni...",[Jos D'Haese]
2,1456916,Werkgroep Institutionele Zaken,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,None,[],[],[],None,[],None,None
3,1332937,"Commissie voor Algemeen Beleid, Financiën, Beg...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Steven Vandeput],[Willem-Frederik Schiltz],[Arnout Coel],[],None,"[Steven Vandeput, Willem-Frederik Schiltz, Arn...","[Filip Brusselmans, Yves Buysse, Stijn De Roo,...",[Jos D'Haese]
4,1332966,"Commissie voor Binnenlands Bestuur, Gelijke Ka...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Kris Van Dijck],[Sam Van Rooy],[Bart Tommelein],[],None,"[Kris Van Dijck, Sam Van Rooy, Bart Tommelein,...","[Adeline Blancquaert, Guy D'haeseleer, Stephan...",[Lise Vandecasteele]
5,1332996,Commissie voor Brussel en de Vlaamse Rand en D...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Mercedes Van Volcem],[Karl Vanlouwe],[Klaas Slootmans],[],None,"[Mercedes Van Volcem, Karl Vanlouwe, Klaas Slo...","[Meyrem Almaci, Karin Brouwers, Filip Brusselm...",None
6,1333030,"Commissie voor Buitenlands Beleid, Europese Aa...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Cathy Coudyser],[Johan Deckmyn],[Tine Van den Brande],[],None,"[Cathy Coudyser, Johan Deckmyn, Tine Van den B...","[Adeline Blancquaert, Jean-Jacques De Gucht, M...",[Jos D'Haese]
7,1333053,"Commissie voor Cultuur, Jeugd, Sport en Media",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Meyrem Almaci],[Marius Meremans],[Karin Brouwers],[],None,"[Meyrem Almaci, Marius Meremans, Karin Brouwer...","[Immanuel De Reuse, Maaike De Rudder, Andries ...",[Onno Vandewalle]
8,1333088,"Commissie voor Economie, Werk, Sociale Economi...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Robrecht Bothuyne],[Imade Annouri],[Andries Gryffroy],[],None,"[Robrecht Bothuyne, Imade Annouri, Andries Gry...","[Steven Coenegrachts, Johan Danen, Annick De R...",[Kim De Witte]
9,1333137,"Commissie voor Landbouw, Visserij en Plattelan...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Dochy],[Joris Nachtergaele],[Emmily Talpe],[],None,"[Bart Dochy, Joris Nachtergaele, Emmily Talpe,...","[Roosmarijn Beckers, Allessia Claes, Steven Co...",None


For some commissions, there does not seem to be much relevant information. For instance for the `Onderzoekscommissie naar de veiligheid in de kinderopvang`, `Werkgroep Institutionele Zaken` en de `Commissie ad hoc`, no permanent members or presidents are registered. Hence, those cannot be taken into account in a meaninful way to assess presence of members. Hence we can drop them out of the data frame and reset the indices. For `Controlecommissie voor Regeringsmededelingen`, no president is registered, but there are however permanent members.

In [22]:
commissions_overview_df[(commissions_overview_df['voorzitter'].isnull()) |
                                       (commissions_overview_df['vaste leden'].apply(len) == 0)]

,commissie.id,commissie.titel,commissie.link,voorzitter,eerste ondervoorzitter,tweede ondervoorzitter,derde ondervoorzitter,vierde ondervoorzitter,vaste leden,plaatsvervangende leden,toegevoegde leden
0,1621329,Onderzoekscommissie naar de veiligheid in de k...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,None,[],[],[],None,[],None,None
2,1456916,Werkgroep Institutionele Zaken,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,None,[],[],[],None,[],None,None
17,1333275,Controlecommissie voor Regeringsmededelingen,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,None,[],[],[],None,"[Allessia Claes, Guy D'haeseleer, Kurt De Loor...","[Robrecht Bothuyne, Johan Danen, Koen Daniëls,...",None
24,1320193,Commissie ad hoc,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,None,[],[],[],None,[],None,None


In [23]:
commissions_overview_df.drop([0, 2, 24], inplace=True)
commissions_overview_df

commissions_overview_df.reset_index(drop = True, inplace = True)
commissions_overview_df

,commissie.id,commissie.titel,commissie.link,voorzitter,eerste ondervoorzitter,tweede ondervoorzitter,derde ondervoorzitter,vierde ondervoorzitter,vaste leden,plaatsvervangende leden,toegevoegde leden
1,1622680,Commissie Vlaams Energie- en Klimaatplan,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Andries Gryffroy],[Bruno Tobback],[Robrecht Bothuyne],[],None,"[Andries Gryffroy, Bruno Tobback, Robrecht Bot...","[Adeline Blancquaert, Bart Claes, Inez De Coni...",[Jos D'Haese]
3,1332937,"Commissie voor Algemeen Beleid, Financiën, Beg...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Steven Vandeput],[Willem-Frederik Schiltz],[Arnout Coel],[],None,"[Steven Vandeput, Willem-Frederik Schiltz, Arn...","[Filip Brusselmans, Yves Buysse, Stijn De Roo,...",[Jos D'Haese]
4,1332966,"Commissie voor Binnenlands Bestuur, Gelijke Ka...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Kris Van Dijck],[Sam Van Rooy],[Bart Tommelein],[],None,"[Kris Van Dijck, Sam Van Rooy, Bart Tommelein,...","[Adeline Blancquaert, Guy D'haeseleer, Stephan...",[Lise Vandecasteele]
5,1332996,Commissie voor Brussel en de Vlaamse Rand en D...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Mercedes Van Volcem],[Karl Vanlouwe],[Klaas Slootmans],[],None,"[Mercedes Van Volcem, Karl Vanlouwe, Klaas Slo...","[Meyrem Almaci, Karin Brouwers, Filip Brusselm...",None
6,1333030,"Commissie voor Buitenlands Beleid, Europese Aa...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Cathy Coudyser],[Johan Deckmyn],[Tine Van den Brande],[],None,"[Cathy Coudyser, Johan Deckmyn, Tine Van den B...","[Adeline Blancquaert, Jean-Jacques De Gucht, M...",[Jos D'Haese]
7,1333053,"Commissie voor Cultuur, Jeugd, Sport en Media",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Meyrem Almaci],[Marius Meremans],[Karin Brouwers],[],None,"[Meyrem Almaci, Marius Meremans, Karin Brouwer...","[Immanuel De Reuse, Maaike De Rudder, Andries ...",[Onno Vandewalle]
8,1333088,"Commissie voor Economie, Werk, Sociale Economi...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Robrecht Bothuyne],[Imade Annouri],[Andries Gryffroy],[],None,"[Robrecht Bothuyne, Imade Annouri, Andries Gry...","[Steven Coenegrachts, Johan Danen, Annick De R...",[Kim De Witte]
9,1333137,"Commissie voor Landbouw, Visserij en Plattelan...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Dochy],[Joris Nachtergaele],[Emmily Talpe],[],None,"[Bart Dochy, Joris Nachtergaele, Emmily Talpe,...","[Roosmarijn Beckers, Allessia Claes, Steven Co...",None
10,1333177,"Commissie voor Leefmilieu, Natuur, Ruimtelijke...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Gwenny De Vroe],[Bruno Tobback],[Inez De Coninck],[],None,"[Gwenny De Vroe, Bruno Tobback, Inez De Coninc...","[Hannes Anaf, Robrecht Bothuyne, Johan Danen, ...",[Jos D'Haese]
11,1333200,Commissie voor Mobiliteit en Openbare Werken,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Claes],[Martine Fournier],[Els Robeyns],[],None,"[Bart Claes, Martine Fournier, Els Robeyns, Im...","[Roosmarijn Beckers, Karin Brouwers, Arnout Co...",[Jos D'Haese]


,commissie.id,commissie.titel,commissie.link,voorzitter,eerste ondervoorzitter,tweede ondervoorzitter,derde ondervoorzitter,vierde ondervoorzitter,vaste leden,plaatsvervangende leden,toegevoegde leden
0,1622680,Commissie Vlaams Energie- en Klimaatplan,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Andries Gryffroy],[Bruno Tobback],[Robrecht Bothuyne],[],None,"[Andries Gryffroy, Bruno Tobback, Robrecht Bot...","[Adeline Blancquaert, Bart Claes, Inez De Coni...",[Jos D'Haese]
1,1332937,"Commissie voor Algemeen Beleid, Financiën, Beg...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Steven Vandeput],[Willem-Frederik Schiltz],[Arnout Coel],[],None,"[Steven Vandeput, Willem-Frederik Schiltz, Arn...","[Filip Brusselmans, Yves Buysse, Stijn De Roo,...",[Jos D'Haese]
2,1332966,"Commissie voor Binnenlands Bestuur, Gelijke Ka...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Kris Van Dijck],[Sam Van Rooy],[Bart Tommelein],[],None,"[Kris Van Dijck, Sam Van Rooy, Bart Tommelein,...","[Adeline Blancquaert, Guy D'haeseleer, Stephan...",[Lise Vandecasteele]
3,1332996,Commissie voor Brussel en de Vlaamse Rand en D...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Mercedes Van Volcem],[Karl Vanlouwe],[Klaas Slootmans],[],None,"[Mercedes Van Volcem, Karl Vanlouwe, Klaas Slo...","[Meyrem Almaci, Karin Brouwers, Filip Brusselm...",None
4,1333030,"Commissie voor Buitenlands Beleid, Europese Aa...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Cathy Coudyser],[Johan Deckmyn],[Tine Van den Brande],[],None,"[Cathy Coudyser, Johan Deckmyn, Tine Van den B...","[Adeline Blancquaert, Jean-Jacques De Gucht, M...",[Jos D'Haese]
5,1333053,"Commissie voor Cultuur, Jeugd, Sport en Media",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Meyrem Almaci],[Marius Meremans],[Karin Brouwers],[],None,"[Meyrem Almaci, Marius Meremans, Karin Brouwer...","[Immanuel De Reuse, Maaike De Rudder, Andries ...",[Onno Vandewalle]
6,1333088,"Commissie voor Economie, Werk, Sociale Economi...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Robrecht Bothuyne],[Imade Annouri],[Andries Gryffroy],[],None,"[Robrecht Bothuyne, Imade Annouri, Andries Gry...","[Steven Coenegrachts, Johan Danen, Annick De R...",[Kim De Witte]
7,1333137,"Commissie voor Landbouw, Visserij en Plattelan...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Dochy],[Joris Nachtergaele],[Emmily Talpe],[],None,"[Bart Dochy, Joris Nachtergaele, Emmily Talpe,...","[Roosmarijn Beckers, Allessia Claes, Steven Co...",None
8,1333177,"Commissie voor Leefmilieu, Natuur, Ruimtelijke...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Gwenny De Vroe],[Bruno Tobback],[Inez De Coninck],[],None,"[Gwenny De Vroe, Bruno Tobback, Inez De Coninc...","[Hannes Anaf, Robrecht Bothuyne, Johan Danen, ...",[Jos D'Haese]
9,1333200,Commissie voor Mobiliteit en Openbare Werken,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Claes],[Martine Fournier],[Els Robeyns],[],None,"[Bart Claes, Martine Fournier, Els Robeyns, Im...","[Roosmarijn Beckers, Karin Brouwers, Arnout Co...",[Jos D'Haese]


## Vergaderingen commissies

First, we obtain the relevant time frame for which we want to obtain attendance information. If we want to assess attendance at all meetings of the current legislature, we can use the `get_endpoint()` function to obtain this.

In [24]:
# use endpoint to obtain information about legislaturen
legislaturen = get_endpoint('/leg/alle')

# Extracting the most recent legislatuur start date
legislatuur_items = legislaturen['items']
most_recent_legislatuur = max(legislatuur_items, key=lambda x: x['legislatuur']['start-legislatuur'])
start_date_most_recent_legislatuur_str = most_recent_legislatuur['legislatuur']['start-legislatuur']
start_date_most_recent_legislatuur_str

start = datetime.strptime(start_date_most_recent_legislatuur_str, '%Y-%m-%dT%H:%M:%S%z').date()
print(f'Starting point of monitoring: {start}.')

'2019-06-18T00:00:00+0200'

Starting point of monitoring: 2019-06-18.


In [25]:
# Set end time
end = datetime.now().date()
print(f'End point of monitoring: {end}.')

# Create string version (to append to filenames of output files)
end_str = end.strftime("%Y-%m-%d")
end_str

End point of monitoring: 2023-12-23.


'2023-12-23'

In [26]:
# # ========================= DEVELOPMENT ==============================================
# # set temporary start time
# start = datetime(2023, 9, 1).date()
# print(f'Starting point of monitoring: {start}.')
# # ========================= DEVELOPMENT ==============================================

Then we obtain all previous meetings for a specific commission, for a certain time frame. First, we create a helper function to extract the meeting id's of all relevant meetings (`extract_previous_meeting_ids_zoek()`). Then we use another helper function to use those meeting id's to extract the attendance information on all those meetings (`extract_meeting_details()`). 

In [27]:
def extract_previous_meeting_ids_zoek(start_date, end_date, commission_id):
    # Convert dates to the required format (ddmmyyyy)
    start_date_str = start_date.strftime("%d%m%Y")
    end_date_str = end_date.strftime("%d%m%Y")

    # API endpoint and parameters
    endpoint = '/verg/zoek/datums'
    params = {
        'type': 'comm',  # Choosing plenaire meetings
        'datumVan': start_date_str,
        'datumTot': end_date_str,
        'idComm': commission_id  # Specific commission ID
    }

    response = requests.get(base_url + endpoint, params=params)

    if response.status_code == 200:
        # Process the response data (e.g., extract meetings)
        meetings = response.json()  # Assuming the response is in JSON format
        
        # Obtain meeting ids of all relevant meetings
        meeting_ids = [item['vergadering']['id'] for item in meetings['items']]
        
        return meeting_ids
    else:
        print(f"Failed to fetch meetings. Status code: {response.status_code}")
        return None

In [28]:
def extract_meeting_details(idVerg:int):
    # API endpoint and parameters
    endpoint = f'/verg/{idVerg}'
    params = {
        'idVerg': idVerg  # Specific meeting ID
    }

    response = requests.get(base_url + endpoint, params=params)

    if response.status_code == 200:
        # Process the response data (e.g., extract meeting details)
        meeting_details = response.json()  # Assuming the response is in JSON format
        return meeting_details

Then we apply both functions to obtain all attendance information for all relevant meetings for all commissions, and store them in a dict.  

In [29]:
# Create empty column to store total of meetings for commission in relevant timeframe
commissions_overview_df["aantal vergaderingen"] = ''

# Create empty dict to store attendance information for all relevant meetings for all commissions 
overall_attendance_dict = {}

# Iterate over each commission
for index_overview, row_overview in commissions_overview_df.iterrows():
    # Obtain commission_id, commission_title and vaste leden
    commission_id = row_overview["commissie.id"]
    commission_title = row_overview["commissie.titel"]
    vaste_leden_spec_comm = row_overview["vaste leden"]
    
    #Show progress
    print("-" * 50)
    print(f"Processing: {commission_id}, {commission_title}")
    
    # For each commisison: obtain the meeting ids of the relevant previous meetings
    previous_meetings_ids = extract_previous_meeting_ids_zoek(start, end, commission_id)
    
    # Create empty dict to store information about the meetings of those meeting ids in
    aanwezigheid_vergaderingen_spec_comm_dict = {}
    
    # For each of the meeting ids: extract the meeting details, and store in dict  
    for idVerg in previous_meetings_ids:
        meeting_details = extract_meeting_details(idVerg)
        
        # Extract date of meeting (us 'datumagendering' and not e.g. 'datumbegin': if meeting cancelled: othterwise key error)
        #meeting_date = meeting_details['vergadering']['commissie'][0]['datumvan']
        meeting_date_str = meeting_details['vergadering']['datumagendering'] 
        meeting_date = datetime.strptime(meeting_date_str, "%Y-%m-%dT%H:%M:%S%z").date()
        
        #Create empty dict to store aanwezigheidsinformatie of specifieke vergadering
        aanwezigheid_spec_verg_dict = {}
        
        # Add date of meeting: convert extracted string to correct format to read as datetime object, and then only extract date()
        aanwezigheid_spec_verg_dict['Datum vergadering'] = meeting_date
        
        # Iterate over the various attendance statuses ('AANWEZIG', 'AFWEZIG', 'VERONTSCHULDIGD') 
        # Add try-except to address KeyErrors about 'aanwezigheid'
        try:    
            for attendance_status in meeting_details['vergadering']['aanwezigheid']:
                status = attendance_status['aanwezigheid-status']
                aanwezigheid_spec_verg_dict[status] = [] #Create empty list for this status to later on fill with members

                # Iterate over all people that are stored under that attendence status and obtain personal information
                for person in attendance_status['persoon']:
                    full_name = f"{person['voornaam']} {person['naam']}"
                    person_info = {
                        'Naam': full_name,
                        'id': person['id'],
                        'Fractie': person['fractie']['naam']
                    }
#                     # Store as list instead of dict to allow easier rendering in Dash
#                     person_info = [full_name, person['id'], person['fractie']['naam']]
                    # Store information in dict of spec meeting
                    aanwezigheid_spec_verg_dict[status].append(person_info)
            # Store information in attandance information dict of all relevant meetings
            aanwezigheid_vergaderingen_spec_comm_dict["Vergadering " + str(idVerg)] = aanwezigheid_spec_verg_dict
        except KeyError:
            print(f'----- No attendance information found for: idVerg: {idVerg}, meeting date: {meeting_date}')
    
        
    
    # Transform data to dataframe for easier handling
    spec_comm_df = pd.DataFrame.from_dict(aanwezigheid_vergaderingen_spec_comm_dict,
                                          orient='index') # use index orientation to get meetings as rows
    
    # Create new empty columns to contain only attendance status for permanent members
    spec_comm_df["AANWEZIG_vast"], spec_comm_df["AFWEZIG_vast"], spec_comm_df["VERONTSCHULDIGD_vast"] = "", "", ""

    # Iterate over each meeting of specific commission and assign the permanent members to the relevant attendance status in new columns
    # Use try-except to handle commissions where no column 'Afwezig' is registered
    for index_spec_comm, row_spec_comm in spec_comm_df.iterrows():
        try:
            spec_comm_df.at[index_spec_comm, "AANWEZIG_vast"] = [member for member in row_spec_comm["AANWEZIG"] if member["Naam"] in vaste_leden_spec_comm] if isinstance(row_spec_comm["AANWEZIG"], list) else []
        except: 
            spec_comm_df.at[index_spec_comm, "AANWEZIG_vast"] = None
            print(f"No column 'AANWEZIG' for {index_overview} {index_spec_comm}.")
        try: 
            spec_comm_df.at[index_spec_comm, "AFWEZIG_vast"] = [member for member in row_spec_comm["AFWEZIG"] if member["Naam"] in vaste_leden_spec_comm] if isinstance(row_spec_comm["AFWEZIG"], list) else []
        except: 
            spec_comm_df.at[index_spec_comm, "AFWEZIG_vast"] = None
            print(f"No column 'AANWEZIG' for {index_overview} {index_spec_comm}.")
        try: 
            spec_comm_df.at[index_spec_comm, "VERONTSCHULDIGD_vast"] = [member for member in row_spec_comm["VERONTSCHULDIGD"] if member["Naam"] in vaste_leden_spec_comm] if isinstance(row_spec_comm["VERONTSCHULDIGD"], list) else []
        except: 
            spec_comm_df.at[index_spec_comm, "VERONTSCHULDIGD_vast"] = None
            print(f"No column 'AANWEZIG' for {index_overview} {index_spec_comm}.")
        
    
    # Add commission name to each row, for easier filtering later on
    spec_comm_df['commissie.titel'] = commission_title
    
    # Store amount of meetings for this commission in main dataframe
    commissions_overview_df.at[index_overview, "aantal vergaderingen"] = spec_comm_df.shape[0]
    
    # Store all attendance information of all meetings of this commission in overall dict for later assessment
    overall_attendance_dict[commission_title] = spec_comm_df
    
    
    #Show progress
    print(f"+++ --> {len(overall_attendance_dict[commission_title])} meetings processed.")
print("-" * 75)
print("--> Processing complete")
print("-" * 75)

--------------------------------------------------
Processing: 1622680, Commissie Vlaams Energie- en Klimaatplan
----- No attendance information found for: idVerg: 1647986, meeting date: 2022-07-14
----- No attendance information found for: idVerg: 1691871, meeting date: 2023-02-10
+++ --> 37 meetings processed.
--------------------------------------------------
Processing: 1332937, Commissie voor Algemeen Beleid, Financiën, Begroting en Justitie
----- No attendance information found for: idVerg: 1379584, meeting date: 2020-03-17
----- No attendance information found for: idVerg: 1384743, meeting date: 2020-04-28
----- No attendance information found for: idVerg: 1385544, meeting date: 2020-05-05
----- No attendance information found for: idVerg: 1388306, meeting date: 2020-05-12
----- No attendance information found for: idVerg: 1389893, meeting date: 2020-05-19
----- No attendance information found for: idVerg: 1391533, meeting date: 2020-05-26
----- No attendance information found f

----- No attendance information found for: idVerg: 1598957, meeting date: 2022-02-03
----- No attendance information found for: idVerg: 1602167, meeting date: 2022-02-10
----- No attendance information found for: idVerg: 1602169, meeting date: 2022-02-10
----- No attendance information found for: idVerg: 1605167, meeting date: 2022-02-16
----- No attendance information found for: idVerg: 1605173, meeting date: 2022-02-17
----- No attendance information found for: idVerg: 1609268, meeting date: 2022-02-23
----- No attendance information found for: idVerg: 1609266, meeting date: 2022-02-24
----- No attendance information found for: idVerg: 1609270, meeting date: 2022-02-24
----- No attendance information found for: idVerg: 1613909, meeting date: 2022-03-10
----- No attendance information found for: idVerg: 1613919, meeting date: 2022-03-10
----- No attendance information found for: idVerg: 1619120, meeting date: 2022-03-25
----- No attendance information found for: idVerg: 1627741, meeti

----- No attendance information found for: idVerg: 1467067, meeting date: 2021-01-28
----- No attendance information found for: idVerg: 1470637, meeting date: 2021-02-04
----- No attendance information found for: idVerg: 1470644, meeting date: 2021-02-04
----- No attendance information found for: idVerg: 1474259, meeting date: 2021-02-11
----- No attendance information found for: idVerg: 1479957, meeting date: 2021-02-23
----- No attendance information found for: idVerg: 1480349, meeting date: 2021-02-25
----- No attendance information found for: idVerg: 1480353, meeting date: 2021-02-25
----- No attendance information found for: idVerg: 1481655, meeting date: 2021-03-03
----- No attendance information found for: idVerg: 1481626, meeting date: 2021-03-04
----- No attendance information found for: idVerg: 1485989, meeting date: 2021-03-11
----- No attendance information found for: idVerg: 1481630, meeting date: 2021-03-11
----- No attendance information found for: idVerg: 1489140, meeti

+++ --> 108 meetings processed.
--------------------------------------------------
Processing: 1333137, Commissie voor Landbouw, Visserij en Plattelandsbeleid
----- No attendance information found for: idVerg: 1346575, meeting date: 2019-12-04
----- No attendance information found for: idVerg: 1364709, meeting date: 2020-02-07
----- No attendance information found for: idVerg: 1374194, meeting date: 2020-03-06
----- No attendance information found for: idVerg: 1378716, meeting date: 2020-03-18
----- No attendance information found for: idVerg: 1382725, meeting date: 2020-04-08
----- No attendance information found for: idVerg: 1383545, meeting date: 2020-04-29
----- No attendance information found for: idVerg: 1385367, meeting date: 2020-05-06
----- No attendance information found for: idVerg: 1387878, meeting date: 2020-05-13
----- No attendance information found for: idVerg: 1390571, meeting date: 2020-05-20
----- No attendance information found for: idVerg: 1391357, meeting date: 20

----- No attendance information found for: idVerg: 1388426, meeting date: 2020-05-12
----- No attendance information found for: idVerg: 1388676, meeting date: 2020-05-13
----- No attendance information found for: idVerg: 1390652, meeting date: 2020-05-20
----- No attendance information found for: idVerg: 1391666, meeting date: 2020-05-26
----- No attendance information found for: idVerg: 1391717, meeting date: 2020-05-27
----- No attendance information found for: idVerg: 1393741, meeting date: 2020-06-02
----- No attendance information found for: idVerg: 1393803, meeting date: 2020-06-03
----- No attendance information found for: idVerg: 1396626, meeting date: 2020-06-09
----- No attendance information found for: idVerg: 1396648, meeting date: 2020-06-10
----- No attendance information found for: idVerg: 1399643, meeting date: 2020-06-16
----- No attendance information found for: idVerg: 1399695, meeting date: 2020-06-17
----- No attendance information found for: idVerg: 1403891, meeti

...

----- No attendance information found for: idVerg: 1474974, meeting date: 2021-02-11
----- No attendance information found for: idVerg: 1480168, meeting date: 2021-02-25
----- No attendance information found for: idVerg: 1480629, meeting date: 2021-03-04
----- No attendance information found for: idVerg: 1480633, meeting date: 2021-03-04
----- No attendance information found for: idVerg: 1485407, meeting date: 2021-03-11
----- No attendance information found for: idVerg: 1485494, meeting date: 2021-03-11
----- No attendance information found for: idVerg: 1485498, meeting date: 2021-03-12
----- No attendance information found for: idVerg: 1489094, meeting date: 2021-03-18
----- No attendance information found for: idVerg: 1489177, meeting date: 2021-03-18
----- No attendance information found for: idVerg: 1491432, meeting date: 2021-03-25
----- No attendance information found for: idVerg: 1491434, meeting date: 2021-03-25
----- No attendance information found for: idVerg: 1496756, meeti

----- No attendance information found for: idVerg: 1743894, meeting date: 2023-06-15
----- No attendance information found for: idVerg: 1750578, meeting date: 2023-06-28
----- No attendance information found for: idVerg: 1763473, meeting date: 2023-10-20
+++ --> 141 meetings processed.
--------------------------------------------------
Processing: 1333260, Commissie voor Welzijn, Volksgezondheid, Gezin en Armoedebestrijding
----- No attendance information found for: idVerg: 1360106, meeting date: 2020-02-12
----- No attendance information found for: idVerg: 1369300, meeting date: 2020-03-04
----- No attendance information found for: idVerg: 1379609, meeting date: 2020-03-17
----- No attendance information found for: idVerg: 1382664, meeting date: 2020-04-07
----- No attendance information found for: idVerg: 1383821, meeting date: 2020-04-21
----- No attendance information found for: idVerg: 1384722, meeting date: 2020-04-28
----- No attendance information found for: idVerg: 1384712, me

----- No attendance information found for: idVerg: 1558970, meeting date: 2021-10-12
----- No attendance information found for: idVerg: 1562810, meeting date: 2021-10-19
----- No attendance information found for: idVerg: 1565388, meeting date: 2021-10-26
----- No attendance information found for: idVerg: 1565405, meeting date: 2021-10-27
----- No attendance information found for: idVerg: 1569382, meeting date: 2021-11-09
----- No attendance information found for: idVerg: 1572748, meeting date: 2021-11-16
----- No attendance information found for: idVerg: 1574412, meeting date: 2021-11-23
----- No attendance information found for: idVerg: 1574437, meeting date: 2021-11-24
----- No attendance information found for: idVerg: 1576400, meeting date: 2021-11-30
----- No attendance information found for: idVerg: 1577982, meeting date: 2021-12-07
----- No attendance information found for: idVerg: 1581273, meeting date: 2021-12-14
----- No attendance information found for: idVerg: 1581303, meeti

----- No attendance information found for: idVerg: 1577440, meeting date: 2021-12-09
----- No attendance information found for: idVerg: 1579075, meeting date: 2021-12-16
----- No attendance information found for: idVerg: 1585613, meeting date: 2021-12-23
----- No attendance information found for: idVerg: 1588738, meeting date: 2022-01-13
----- No attendance information found for: idVerg: 1613392, meeting date: 2022-03-10
----- No attendance information found for: idVerg: 1673818, meeting date: 2022-10-24
----- No attendance information found for: idVerg: 1691056, meeting date: 2022-12-22
----- No attendance information found for: idVerg: 1757137, meeting date: 2023-07-13
----- No attendance information found for: idVerg: 1770863, meeting date: 2023-10-18
+++ --> 68 meetings processed.
--------------------------------------------------
Processing: 1333268, Commissie voor Reglement en Samenwerking
----- No attendance information found for: idVerg: 1375866, meeting date: 2020-03-17
+++ --

----- No attendance information found for: idVerg: 1540569, meeting date: 2021-07-02
----- No attendance information found for: idVerg: 1541393, meeting date: 2021-07-09
----- No attendance information found for: idVerg: 1541972, meeting date: 2021-07-09
----- No attendance information found for: idVerg: 1545091, meeting date: 2021-07-16
----- No attendance information found for: idVerg: 1545104, meeting date: 2021-07-16
----- No attendance information found for: idVerg: 1548095, meeting date: 2021-08-16
----- No attendance information found for: idVerg: 1548098, meeting date: 2021-08-20
----- No attendance information found for: idVerg: 1548244, meeting date: 2021-08-27
----- No attendance information found for: idVerg: 1548251, meeting date: 2021-08-27
----- No attendance information found for: idVerg: 1548455, meeting date: 2021-08-30
----- No attendance information found for: idVerg: 1548466, meeting date: 2021-09-03
----- No attendance information found for: idVerg: 1548766, meeti

In [30]:
# Inspect results
overall_attendance_dict.keys()
overall_attendance_dict['Commissie voor Cultuur, Jeugd, Sport en Media']

commissions_overview_df[["commissie.titel", "aantal vergaderingen"]]

dict_keys(['Commissie Vlaams Energie- en Klimaatplan', 'Commissie voor Algemeen Beleid, Financiën, Begroting en Justitie', 'Commissie voor Binnenlands Bestuur, Gelijke Kansen en Inburgering', 'Commissie voor Brussel en de Vlaamse Rand en Dierenwelzijn', 'Commissie voor Buitenlands Beleid, Europese Aangelegenheden, Internationale Samenwerking en Toerisme', 'Commissie voor Cultuur, Jeugd, Sport en Media', 'Commissie voor Economie, Werk, Sociale Economie, Wetenschap en Innovatie', 'Commissie voor Landbouw, Visserij en Plattelandsbeleid', 'Commissie voor Leefmilieu, Natuur, Ruimtelijke Ordening en Energie', 'Commissie voor Mobiliteit en Openbare Werken', 'Commissie voor Onderwijs', 'Commissie voor Welzijn, Volksgezondheid, Gezin en Armoedebestrijding', 'Commissie voor Wonen en Onroerend Erfgoed', 'Commissie voor Reglement en Samenwerking', 'Commissie voor de Vervolgingen', 'Controlecommissie voor Regeringsmededelingen', 'Deontologische Commissie', 'Vlaamse Controlecommissie voor de Verkiez

,Datum vergadering,AANWEZIG,AFWEZIG,VERONTSCHULDIGD,AANWEZIG_vast,AFWEZIG_vast,VERONTSCHULDIGD_vast,commissie.titel
Vergadering 1333440,2019-10-16,"[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Cathy Coudyser', 'id': 4068, 'Fract...",NaN,"[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Cathy Coudyser', 'id': 4068, 'Fract...",[],"Commissie voor Cultuur, Jeugd, Sport en Media"
Vergadering 1333447,2019-10-17,"[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Johan Deckmyn', 'id': 2838, 'Fracti...",NaN,"[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Johan Deckmyn', 'id': 2838, 'Fracti...",[],"Commissie voor Cultuur, Jeugd, Sport en Media"
Vergadering 1335863,2019-10-23,"[{'Naam': 'Els Ampe', 'id': 4382, 'Fractie': '...","[{'Naam': 'Karin Brouwers', 'id': 3426, 'Fract...","[{'Naam': 'Manuela Van Werde', 'id': 4099, 'Fr...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Karin Brouwers', 'id': 3426, 'Fract...","[{'Naam': 'Manuela Van Werde', 'id': 4099, 'Fr...","Commissie voor Cultuur, Jeugd, Sport en Media"
Vergadering 1335873,2019-10-24,"[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Filip Brusselmans', 'id': 4387, 'Fr...","[{'Naam': 'Manuela Van Werde', 'id': 4099, 'Fr...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Filip Brusselmans', 'id': 4387, 'Fr...","[{'Naam': 'Manuela Van Werde', 'id': 4099, 'Fr...","Commissie voor Cultuur, Jeugd, Sport en Media"
Vergadering 1335875,2019-10-24,"[{'Naam': 'Karin Brouwers', 'id': 3426, 'Fract...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...",NaN,"[{'Naam': 'Karin Brouwers', 'id': 3426, 'Fract...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...",[],"Commissie voor Cultuur, Jeugd, Sport en Media"
...,...,...,...,...,...,...,...,...
Vergadering 1782195,2023-11-30,"[{'Naam': 'Meyrem Almaci', 'id': 4381, 'Fracti...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Stephanie D'Hose', 'id': 4395, 'Fra...","[{'Naam': 'Meyrem Almaci', 'id': 4381, 'Fracti...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Stephanie D'Hose', 'id': 4395, 'Fra...","Commissie voor Cultuur, Jeugd, Sport en Media"
Vergadering 1784035,2023-12-07,"[{'Naam': 'Meyrem Almaci', 'id': 4381, 'Fracti...","[{'Naam': 'Johan Deckmyn', 'id': 2838, 'Fracti...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Meyrem Almaci', 'id': 4381, 'Fracti...","[{'Naam': 'Johan Deckmyn', 'id': 2838, 'Fracti...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","Commissie voor Cultuur, Jeugd, Sport en Media"
Vergadering 1784043,2023-12-07,"[{'Naam': 'Meyrem Almaci', 'id': 4381, 'Fracti...","[{'Naam': 'Steven Coenegrachts', 'id': 4446, '...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Meyrem Almaci', 'id': 4381, 'Fracti...","[{'Naam': 'Steven Coenegrachts', 'id': 4446, '...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","Commissie voor Cultuur, Jeugd, Sport en Media"
Vergadering 1786243,2023-12-14,"[{'Naam': 'Meyrem Almaci', 'id': 4381, 'Fracti...","[{'Naam': 'Johan Deckmyn', 'id': 2838, 'Fracti...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Meyrem Almaci', 'id': 4381, 'Fracti...","[{'Naam': 'Johan Deckmyn', 'id': 2838, 'Fracti...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","Commissie voor Cultuur, Jeugd, Sport en Media"


,commissie.titel,aantal vergaderingen
0,Commissie Vlaams Energie- en Klimaatplan,37
1,"Commissie voor Algemeen Beleid, Financiën, Beg...",81
2,"Commissie voor Binnenlands Bestuur, Gelijke Ka...",84
3,Commissie voor Brussel en de Vlaamse Rand en D...,51
4,"Commissie voor Buitenlands Beleid, Europese Aa...",71
5,"Commissie voor Cultuur, Jeugd, Sport en Media",146
6,"Commissie voor Economie, Werk, Sociale Economi...",108
7,"Commissie voor Landbouw, Visserij en Plattelan...",70
8,"Commissie voor Leefmilieu, Natuur, Ruimtelijke...",131
9,Commissie voor Mobiliteit en Openbare Werken,76


In [31]:
# Already extract all meetings of all commissions, and store in dataframe
meetings_all_commissions_df = pd.concat([overall_attendance_dict[key] for key in overall_attendance_dict.keys()])

# Inspect results
meetings_all_commissions_df.shape

(1216, 8)

Then we further modify the resulting dataframes. First we create some empty columns that will be later on filled during the interactive dash application. Some columns (i.e. those in `meetings_all_commissions_df` indicating how many members were present/absent/absent with notice, can however already be filled.  

In [32]:
# Create empty columns in commissions_overview_df to fill later on using .loc
columns_to_fill = [
    'aanwezig_count_alle', 'afwezig_count_alle', 'verontschuldigd_count_alle',
    'aanwezig_count_vaste', 'afwezig_count_vaste', 'verontschuldigd_count_vaste',
    'Gemiddelde aantal aanwezig alle leden', 'Gemiddelde aantal afwezig alle leden', 
    'Gemiddelde aantal verontschuldigd alle leden',
    'Gemiddelde aantal aanwezig vaste leden', 'Gemiddelde aantal afwezig vaste leden',
    'Gemiddelde aantal verontschuldigd vaste leden']

# Assign empty strings and NaN values explicitly using .loc
for column in columns_to_fill:
    commissions_overview_df.loc[:, column] = "" if 'count' in column else np.nan

In [33]:
# Create empty columns in meetings_all_commissions_filtered_df
new_columns = ["Aantal aanwezig alle leden", "Aantal afwezig alle leden", "Aantal verontschuldigd alle leden",
    "Aantal aanwezig vaste leden", "Aantal afwezig vaste leden", "Aantal verontschuldigd vaste leden"]

# Assign np.nan to the new columns
for new_col in new_columns:
    meetings_all_commissions_df[new_col] = np.nan

# Define a function to get the count for each column
def get_count(column):
    return len(column) if isinstance(column, list) else 0

# Modify meetings_all_commissions_filtered_df with counts for attendance statuses 
for index_spec, row_specific in meetings_all_commissions_df.iterrows():
    for column_name_spec_to_fill, column_name_to_calculate_from in zip(
        ["Aantal aanwezig alle leden", "Aantal afwezig alle leden", "Aantal verontschuldigd alle leden",
         "Aantal aanwezig vaste leden", "Aantal afwezig vaste leden", "Aantal verontschuldigd vaste leden"],
        ['AANWEZIG', 'AFWEZIG', 'VERONTSCHULDIGD','AANWEZIG_vast', 'AFWEZIG_vast', 'VERONTSCHULDIGD_vast']):
        meetings_all_commissions_df.at[index_spec, column_name_spec_to_fill] = get_count(
            meetings_all_commissions_df.loc[index_spec, column_name_to_calculate_from])

In [34]:
# Inspect results
commissions_overview_df
meetings_all_commissions_df

,commissie.id,commissie.titel,commissie.link,voorzitter,eerste ondervoorzitter,tweede ondervoorzitter,derde ondervoorzitter,vierde ondervoorzitter,vaste leden,plaatsvervangende leden,...,verontschuldigd_count_alle,aanwezig_count_vaste,afwezig_count_vaste,verontschuldigd_count_vaste,Gemiddelde aantal aanwezig alle leden,Gemiddelde aantal afwezig alle leden,Gemiddelde aantal verontschuldigd alle leden,Gemiddelde aantal aanwezig vaste leden,Gemiddelde aantal afwezig vaste leden,Gemiddelde aantal verontschuldigd vaste leden
0,1622680,Commissie Vlaams Energie- en Klimaatplan,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Andries Gryffroy],[Bruno Tobback],[Robrecht Bothuyne],[],None,"[Andries Gryffroy, Bruno Tobback, Robrecht Bot...","[Adeline Blancquaert, Bart Claes, Inez De Coni...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
1,1332937,"Commissie voor Algemeen Beleid, Financiën, Beg...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Steven Vandeput],[Willem-Frederik Schiltz],[Arnout Coel],[],None,"[Steven Vandeput, Willem-Frederik Schiltz, Arn...","[Filip Brusselmans, Yves Buysse, Stijn De Roo,...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
2,1332966,"Commissie voor Binnenlands Bestuur, Gelijke Ka...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Kris Van Dijck],[Sam Van Rooy],[Bart Tommelein],[],None,"[Kris Van Dijck, Sam Van Rooy, Bart Tommelein,...","[Adeline Blancquaert, Guy D'haeseleer, Stephan...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
3,1332996,Commissie voor Brussel en de Vlaamse Rand en D...,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Mercedes Van Volcem],[Karl Vanlouwe],[Klaas Slootmans],[],None,"[Mercedes Van Volcem, Karl Vanlouwe, Klaas Slo...","[Meyrem Almaci, Karin Brouwers, Filip Brusselm...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
4,1333030,"Commissie voor Buitenlands Beleid, Europese Aa...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Cathy Coudyser],[Johan Deckmyn],[Tine Van den Brande],[],None,"[Cathy Coudyser, Johan Deckmyn, Tine Van den B...","[Adeline Blancquaert, Jean-Jacques De Gucht, M...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
5,1333053,"Commissie voor Cultuur, Jeugd, Sport en Media",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Meyrem Almaci],[Marius Meremans],[Karin Brouwers],[],None,"[Meyrem Almaci, Marius Meremans, Karin Brouwer...","[Immanuel De Reuse, Maaike De Rudder, Andries ...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
6,1333088,"Commissie voor Economie, Werk, Sociale Economi...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Robrecht Bothuyne],[Imade Annouri],[Andries Gryffroy],[],None,"[Robrecht Bothuyne, Imade Annouri, Andries Gry...","[Steven Coenegrachts, Johan Danen, Annick De R...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
7,1333137,"Commissie voor Landbouw, Visserij en Plattelan...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Dochy],[Joris Nachtergaele],[Emmily Talpe],[],None,"[Bart Dochy, Joris Nachtergaele, Emmily Talpe,...","[Roosmarijn Beckers, Allessia Claes, Steven Co...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
8,1333177,"Commissie voor Leefmilieu, Natuur, Ruimtelijke...",[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Gwenny De Vroe],[Bruno Tobback],[Inez De Coninck],[],None,"[Gwenny De Vroe, Bruno Tobback, Inez De Coninc...","[Hannes Anaf, Robrecht Bothuyne, Johan Danen, ...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN
9,1333200,Commissie voor Mobiliteit en Openbare Werken,[{'href': 'https://ws.vlpar.be/e/opendata/comm...,[Bart Claes],[Martine Fournier],[Els Robeyns],[],None,"[Bart Claes, Martine Fournier, Els Robeyns, Im...","[Roosmarijn Beckers, Karin Brouwers, Arnout Co...",...,,,,,NaN,NaN,NaN,NaN,NaN,NaN


,Datum vergadering,AANWEZIG,AFWEZIG,VERONTSCHULDIGD,AANWEZIG_vast,AFWEZIG_vast,VERONTSCHULDIGD_vast,commissie.titel,Aantal aanwezig alle leden,Aantal afwezig alle leden,Aantal verontschuldigd alle leden,Aantal aanwezig vaste leden,Aantal afwezig vaste leden,Aantal verontschuldigd vaste leden
Vergadering 1622819,2022-04-01,"[{'Naam': 'Staf Aerts', 'id': 4544, 'Fractie':...","[{'Naam': 'Rita Moors', 'id': 4524, 'Fractie':...",NaN,"[{'Naam': 'Staf Aerts', 'id': 4544, 'Fractie':...","[{'Naam': 'Rita Moors', 'id': 4524, 'Fractie':...",[],Commissie Vlaams Energie- en Klimaatplan,13.0,4.0,0.0,12.0,4.0,0.0
Vergadering 1624832,2022-04-22,"[{'Naam': 'Staf Aerts', 'id': 4544, 'Fractie':...","[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...",NaN,"[{'Naam': 'Staf Aerts', 'id': 4544, 'Fractie':...","[{'Naam': 'Arnout Coel', 'id': 4444, 'Fractie'...",[],Commissie Vlaams Energie- en Klimaatplan,10.0,6.0,0.0,10.0,6.0,0.0
Vergadering 1628257,2022-04-29,"[{'Naam': 'Staf Aerts', 'id': 4544, 'Fractie':...","[{'Naam': 'Peter Van Rompuy', 'id': 3918, 'Fra...","[{'Naam': 'Philippe Muyters', 'id': 3461, 'Fra...","[{'Naam': 'Staf Aerts', 'id': 4544, 'Fractie':...","[{'Naam': 'Peter Van Rompuy', 'id': 3918, 'Fra...","[{'Naam': 'Philippe Muyters', 'id': 3461, 'Fra...",Commissie Vlaams Energie- en Klimaatplan,13.0,2.0,1.0,12.0,2.0,1.0
Vergadering 1629767,2022-05-06,"[{'Naam': 'Staf Aerts', 'id': 4544, 'Fractie':...","[{'Naam': 'Steven Coenegrachts', 'id': 4446, '...","[{'Naam': 'Allessia Claes', 'id': 4389, 'Fract...","[{'Naam': 'Staf Aerts', 'id': 4544, 'Fractie':...","[{'Naam': 'Steven Coenegrachts', 'id': 4446, '...","[{'Naam': 'Allessia Claes', 'id': 4389, 'Fract...",Commissie Vlaams Energie- en Klimaatplan,11.0,3.0,2.0,10.0,3.0,2.0
Vergadering 1630766,2022-05-13,"[{'Naam': 'Staf Aerts', 'id': 4544, 'Fractie':...","[{'Naam': 'Peter Van Rompuy', 'id': 3918, 'Fra...","[{'Naam': 'Philippe Muyters', 'id': 3461, 'Fra...","[{'Naam': 'Staf Aerts', 'id': 4544, 'Fractie':...","[{'Naam': 'Peter Van Rompuy', 'id': 3918, 'Fra...","[{'Naam': 'Philippe Muyters', 'id': 3461, 'Fra...",Commissie Vlaams Energie- en Klimaatplan,13.0,2.0,1.0,12.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vergadering 1619010,2022-03-21,"[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Imade Annouri', 'id': 4052, 'Fracti...","[{'Naam': 'Gwenny De Vroe', 'id': 3448, 'Fract...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Imade Annouri', 'id': 4052, 'Fracti...","[{'Naam': 'Gwenny De Vroe', 'id': 3448, 'Fract...",Onderzoekscommissie PFAS-PFOS,8.0,6.0,2.0,8.0,6.0,2.0
Vergadering 1619944,2022-03-24,"[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Imade Annouri', 'id': 4052, 'Fracti...","[{'Naam': 'Joris Nachtergaele', 'id': 4442, 'F...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Imade Annouri', 'id': 4052, 'Fracti...","[{'Naam': 'Joris Nachtergaele', 'id': 4442, 'F...",Onderzoekscommissie PFAS-PFOS,9.0,6.0,1.0,9.0,6.0,1.0
Vergadering 1619013,2022-03-25,"[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Imade Annouri', 'id': 4052, 'Fracti...","[{'Naam': 'Piet De Bruyn', 'id': 3324, 'Fracti...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Imade Annouri', 'id': 4052, 'Fracti...","[{'Naam': 'Piet De Bruyn', 'id': 3324, 'Fracti...",Onderzoekscommissie PFAS-PFOS,9.0,3.0,4.0,9.0,3.0,4.0
Vergadering 1619020,2022-03-25,"[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Imade Annouri', 'id': 4052, 'Fracti...","[{'Naam': 'Piet De Bruyn', 'id': 3324, 'Fracti...","[{'Naam': 'Hannes Anaf', 'id': 4383, 'Fractie'...","[{'Naam': 'Imade Annouri', 'id': 4052, 'Fracti...","[{'Naam': 'Piet De Bruyn', 'id': 3324, 'Fracti...",Onderzoekscommissie PFAS-PFOS,9.0,3.0,4.0,9.0,3.0,4.0


In [35]:
# Extract a version of the dataframe that only contains the names of the members (i.e. the third element)
# Obtain copy of relevant dataframe
meetings_all_commissions_short_df = copy.deepcopy(meetings_all_commissions_df)
# Define the columns to modify
columns_to_modify = [col for col in ['AANWEZIG', 'AFWEZIG', 'VERONTSCHULDIGD','AANWEZIG_vast', 'AFWEZIG_vast', 'VERONTSCHULDIGD_vast']]


# Apply function to modify columns
for col in columns_to_modify:
    meetings_all_commissions_short_df[col] = meetings_all_commissions_short_df[col].apply(
        lambda x: [item["Naam"] for item in x] if isinstance(x, list) else None)

# Inspect results
meetings_all_commissions_short_df

,Datum vergadering,AANWEZIG,AFWEZIG,VERONTSCHULDIGD,AANWEZIG_vast,AFWEZIG_vast,VERONTSCHULDIGD_vast,commissie.titel,Aantal aanwezig alle leden,Aantal afwezig alle leden,Aantal verontschuldigd alle leden,Aantal aanwezig vaste leden,Aantal afwezig vaste leden,Aantal verontschuldigd vaste leden
Vergadering 1622819,2022-04-01,"[Staf Aerts, Robrecht Bothuyne, Allessia Claes...","[Rita Moors, Leo Pieters, Chris Steenwegen, Pe...",None,"[Staf Aerts, Robrecht Bothuyne, Allessia Claes...","[Rita Moors, Leo Pieters, Chris Steenwegen, Pe...",[],Commissie Vlaams Energie- en Klimaatplan,13.0,4.0,0.0,12.0,4.0,0.0
Vergadering 1624832,2022-04-22,"[Staf Aerts, Robrecht Bothuyne, Allessia Claes...","[Arnout Coel, Steven Coenegrachts, Leo Pieters...",None,"[Staf Aerts, Robrecht Bothuyne, Allessia Claes...","[Arnout Coel, Steven Coenegrachts, Leo Pieters...",[],Commissie Vlaams Energie- en Klimaatplan,10.0,6.0,0.0,10.0,6.0,0.0
Vergadering 1628257,2022-04-29,"[Staf Aerts, Robrecht Bothuyne, Allessia Claes...","[Peter Van Rompuy, Wim Verheyden]",[Philippe Muyters],"[Staf Aerts, Robrecht Bothuyne, Allessia Claes...","[Peter Van Rompuy, Wim Verheyden]",[Philippe Muyters],Commissie Vlaams Energie- en Klimaatplan,13.0,2.0,1.0,12.0,2.0,1.0
Vergadering 1629767,2022-05-06,"[Staf Aerts, Robrecht Bothuyne, Arnout Coel, S...","[Steven Coenegrachts, Peter Van Rompuy, Sam Va...","[Allessia Claes, Rita Moors]","[Staf Aerts, Robrecht Bothuyne, Arnout Coel, A...","[Steven Coenegrachts, Peter Van Rompuy, Sam Va...","[Allessia Claes, Rita Moors]",Commissie Vlaams Energie- en Klimaatplan,11.0,3.0,2.0,10.0,3.0,2.0
Vergadering 1630766,2022-05-13,"[Staf Aerts, Robrecht Bothuyne, Allessia Claes...","[Peter Van Rompuy, Sam Van Rooy]",[Philippe Muyters],"[Staf Aerts, Robrecht Bothuyne, Allessia Claes...","[Peter Van Rompuy, Sam Van Rooy]",[Philippe Muyters],Commissie Vlaams Energie- en Klimaatplan,13.0,2.0,1.0,12.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vergadering 1619010,2022-03-21,"[Hannes Anaf, Bart Claes, Jos D'Haese, Koen Da...","[Imade Annouri, Piet De Bruyn, Chris Janssens,...","[Gwenny De Vroe, Andries Gryffroy]","[Hannes Anaf, Bart Claes, Jos D'Haese, Koen Da...","[Imade Annouri, Piet De Bruyn, Chris Janssens,...","[Gwenny De Vroe, Andries Gryffroy]",Onderzoekscommissie PFAS-PFOS,8.0,6.0,2.0,8.0,6.0,2.0
Vergadering 1619944,2022-03-24,"[Hannes Anaf, Bart Claes, Jos D'Haese, Koen Da...","[Imade Annouri, Piet De Bruyn, Gwenny De Vroe,...",[Joris Nachtergaele],"[Hannes Anaf, Bart Claes, Jos D'Haese, Koen Da...","[Imade Annouri, Piet De Bruyn, Gwenny De Vroe,...",[Joris Nachtergaele],Onderzoekscommissie PFAS-PFOS,9.0,6.0,1.0,9.0,6.0,1.0
Vergadering 1619013,2022-03-25,"[Hannes Anaf, Bart Claes, Jos D'Haese, Koen Da...","[Imade Annouri, Gwenny De Vroe, Chris Janssens]","[Piet De Bruyn, Andries Gryffroy, Joris Nachte...","[Hannes Anaf, Bart Claes, Jos D'Haese, Koen Da...","[Imade Annouri, Gwenny De Vroe, Chris Janssens]","[Piet De Bruyn, Andries Gryffroy, Joris Nachte...",Onderzoekscommissie PFAS-PFOS,9.0,3.0,4.0,9.0,3.0,4.0
Vergadering 1619020,2022-03-25,"[Hannes Anaf, Bart Claes, Jos D'Haese, Koen Da...","[Imade Annouri, Gwenny De Vroe, Chris Janssens]","[Piet De Bruyn, Andries Gryffroy, Joris Nachte...","[Hannes Anaf, Bart Claes, Jos D'Haese, Koen Da...","[Imade Annouri, Gwenny De Vroe, Chris Janssens]","[Piet De Bruyn, Andries Gryffroy, Joris Nachte...",Onderzoekscommissie PFAS-PFOS,9.0,3.0,4.0,9.0,3.0,4.0


Then we round the presence amounts to integers, taking into account the original total. This avoids that the runding results for some commissions in 16 or 14 members instaed of 15.

In [38]:
# ## Save dict with all dataframes for later use
# # 1. Save as pkl. 
# with open(f'../data/vergaderingen_commissies/overall_attendance_dict_{today_str}.pkl', 'wb') as file:
#     pickle.dump(overall_attendance_dict, file)

# # 2. Save as xlsx (for easier visual inspection)
# # Create a Pandas Excel writer using xlsxwriter as the engine
# with pd.ExcelWriter(f'../data/vergaderingen_commissies/overall_attendance_dict_{today_str}.xlsx', engine='xlsxwriter') as writer:
#     # Loop through each key-value pair in the dictionary
#     for key, df in overall_attendance_dict.items():
#         # Write each DataFrame to a specific sheet in the Excel file
#         # Limit Excel worksheet name to 30 chars, else error
#         df.to_excel(writer, sheet_name=key[:31], index=False)

In [39]:
## Save meetings_all_commissions_df for later use
# 1. Save as pkl
with open(f'../data/meetings_all_commissions_df_{today_str}.pkl', 'wb') as file:
    pickle.dump(meetings_all_commissions_df, file)

# 2. Save as csv
meetings_all_commissions_df.to_csv(path_or_buf = f'../data/meetings_all_commissions_df_{today_str}.csv',
                               sep = ";",
                               encoding = "utf-16", # to ensure trema's are well handled (e.g. Koen Daniëls)
                               index = False)

In [40]:
## Save meetings_all_commissions_short_df for later use
# 1. Save as pkl
with open(f'../data/meetings_all_commissions_short_df_{today_str}.pkl', 'wb') as file:
    pickle.dump(meetings_all_commissions_short_df, file)

# 2. Save as csv
meetings_all_commissions_short_df.to_csv(path_or_buf = f'../data/meetings_all_commissions_short_df_{today_str}.csv',
                                         sep = ";",
                                         encoding = "utf-16", # to ensure trema's are well handled (e.g. Koen Daniëls)
                                         index = False)

In [41]:
## Save commissions_overview_df for later use
# 1. Save as pkl
with open(f'../data/commissions_overview_df_{today_str}.pkl', 'wb') as file:
    pickle.dump(commissions_overview_df, file)

# 2. Save as csv
commissions_overview_df.to_csv(path_or_buf = f'../data/commissions_overview_df_{today_str}.csv',
                               sep = ";",
                               encoding = "utf-16", # to ensure trema's are well handled (e.g. Koen Daniëls)
                               index = False)

# Test chunks

In [ ]:
# meetings_all_commissions_df.loc["Vergadering 1764019", "AANWEZIG_vast"][0]
# type(meetings_all_commissions_df.loc["Vergadering 1764019", "AANWEZIG_vast"][0])

In [ ]:
# meetings_all_commissions_df["AANWEZIG_vast"].apply(
#     lambda x: [item["Naam"] for item in x] if isinstance(item, list) else None
# )